# Signal Analyzer: Brake Measurement

The measurement is recorded by the brake control unit with the software transient recoder.

## Import

In [ ]:
from __future__ import annotations

In [ ]:
import math

In [ ]:
from dataclasses import (dataclass, field, fields, asdict)

In [ ]:
from itertools import combinations, chain, permutations, repeat, tee

In [ ]:
from typing import (NamedTuple)

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import ipywidgets as widgets

In [ ]:
import plotly.graph_objects as go

In [ ]:
import plotly.express as px

In [ ]:
import plotly.figure_factory as ff

In [ ]:
from plotly.subplots import make_subplots

In [ ]:
from signalyzer import *

## Import Recording

In [ ]:
df = pd.read_csv('./analyzer/recording.csv', delimiter=',', header=[0], skiprows=[1])

### Data Frame of the Recording

In [ ]:
df

### Number of Rows & Columns

In [ ]:
df.shape

### Column Names

In [ ]:
df.columns

### Column Data Types

In [ ]:
df.dtypes

### Row Indexes

In [ ]:
rows = list(range(df.shape[0]))

### Samping Time of the Recording

In [ ]:
dt = 0.02

## Analysis: Measurement

In [ ]:
@dataclass(eq=True, order=True)
class Measurement:
    """ Measurement results. """
    #: sampling time of the measurement in seconds
    dt: float = 0.0
    #: start-point of the measurement
    p1: Point2D = Point2D(0, 0.0)
    #: end-point of the measurement 
    p2: Point2D = Point2D(0, 0.0)
    #: x-interval of the measurement
    dx: int = 0
    #: y-interval of the measurement
    dy: float = 0.0
    #: slope of the measurement interval
    slope: float = 0.0
    #: measurement time
    t: float = 0.0
    #: speed at measurement start-point in [km/h].
    v1: float = 0.0
    #: speed at measurement end-point in [km/h].
    v2: float = 0.0
    #: average acceleration over the measurement time in [m/s^2]        
    a_mean: float = 0.0
    #: linear approximated speed trace of the measurement
    speed: Trace = field(default_factory=Trace)

    @classmethod    
    def from_recording(cls, speed, pressure, **kwargs):
        """ Evaluates the measurement.
    
        :param float dt: sampling time in seconds
        :param speed: list of samples of the speed signal in [km/h]
        :param pressure: list of samples of the pressure signal in [bar]
        :keyword float v0: Speed limit for standstill in [km/h]
        :keyword int start: index of the start-point of the measurement (optional)
        :keyword int stop: index of the end-point of the measurement (optional)
        """
        # measurement interval
        start = kwargs.get('start', -1)
        stop = kwargs.get('stop', -1)

        # find brake-application point
        start = start if 0 <= start < len(speed) else -1
        if start < 0:
            for i, p in enumerate(pressure):
                start = i
                stop = start if stop < start else stop
                if p > 0:
                    break

        # find stopping point
        if stop <= start:
            for i, v in enumerate(speed[start:]):
                stop = i + start
                if v <= kwargs.get('v0', 0.25):
                    break
                
        # measurement interval found?
        if stop > start:
            p1=Point2D(start, speed[start])
            p2=Point2D(stop, speed[stop])
        
            # linear approximation of the vehicle speed
            vehicle_speed = [speed[p1.x] - (x - p1.x) * (p1.y - p2.y) / (p2.x - p1.x) if 
                             p1.x < x <= p2.x else y for x, y in enumerate(speed)]
        
            return cls(
                dt=dt,
                p1=p1,
                p2=p2,
                dx=p2.x - p1.x,
                dy=p1.y - p2.y,
                slope=(p1.y - p2.y) / (p2.x - p1.x), 
                t=(p2.x - p1.x) * dt,
                v1=p1.y,
                v2=p2.y,
                a_mean=(p2.y - p1.y) / (p2.x - p1.x) / (3.6 * dt), 
                speed=Trace('speed:approximate', vehicle_speed))
        else:
            return cls(dt=dt)

In [ ]:
measurement = Measurement.from_recording(dt=0.02, speed=df['$VREF'], pressure=df['$PBZI'], start=400, v0=0.5)

### Start-Point

In [ ]:
measurement.p1

### End-Point

In [ ]:
measurement.p2

### Intervals

In [ ]:
measurement.dx

In [ ]:
measurement.dy

### Slope

In [ ]:
measurement.slope

### Time

In [ ]:
measurement.t

### Mean Acceleration

In [ ]:
measurement.a_mean

## Uni-Variate Kalman-Filter

In [ ]:
class Gaussian(NamedTuple):
    """ Gaussian distribution. """
    #: Mean of the gaussian distribution
    mean : float = 0.0
    #: Variance of the gaussian distribution
    var : float = 0.0

In [ ]:
@dataclass
class KalmanFilter:
    """ Uni-variate Kalman filter. """
    #: Predicted system state N(x, P)  .
    predicted: Trace = field(default_factory=Trace)
    #: Estimated system state N(x, P).
    estimated: Trace = field(default_factory=Trace)
    
    def __post_init__(self):
        self.predicted.label = 'kf:predicted'
        self.estimated.label = 'kf:estimated'
        
    @staticmethod
    def predictor(system: Gaussian, process: Gaussian) -> Gaussian:
        """ Prediction of the observed system.

        :param system: :math:`N(x_{k-1}, P_{k-1})`, the state and variance of the observed system.
        :param process: :math:`N(u, Q)`, the process and process noise 
            to predict the next state and variance of the observed system.

        :returns: :math:`N(x_k, P_k)`, the predicted state and variance of the observed system
            prior the measurement update.
        """
        # system state, system variance
        x, P = system 
        # process model, process noise (variance) 
        u, Q = process 
    
        # predicted system state
        x = x + u
        # predicted system variance
        P = P + Q
    
        return Gaussian(x, P)
    
    @staticmethod
    def corrector(system: Gaussian, measurement: Gaussian) -> Gaussian:
        """ Correction of the observed system.

        :param system: :math:`N(x_k, P_k)`, the predicted state and variance of the observed system. 
        :param measurement: :math:`N(z, R)`, the measurement and measurement variance
            to update the state and variance of the observed sytem

        :returns: :math:`N(x_{k}, P_{k})`, the estimated state and variance of the observed system
            posterior the measurement update.
        """
        # predicted system state, predicted system variance
        x, P = system  
        # measured system state, measurement variance
        z, R = measurement
    
        # residual between measured system state and predicted system state
        y = z - x        

        # kalman filter gain
        K = P / (P + R)  

        # estimated system state
        x = x + K * y      
        # estimated system variance
        P = (1 - K) * P  
    
        return Gaussian(x, P)
    
    def filter(self, z: Trace, R: float, system: Gaussian, process: Gaussian) -> KalmanFilter:      
        # clear traces
        self.predicted.samples.clear()
        self.estimated.samples.clear()

        for _z in z.samples:
            # prediction
            system = self.predictor(system, process)
            self.predicted.samples.append(system[0])
            # estimation
            system = self.corrector(system, Gaussian(_z, R))  
            self.estimated.samples.append(system[0])
            
        return self

## Signal Analysis: Vehicle Velocity

In [ ]:
signal = Trace.from_dict('$VAG1', df)

In [ ]:
# jerk limit [m/s^3]
j = 1.5
# acceleration change rate in m/s^2
da = j * dt
# speed change rate in km/h
dv = da * dt * 3.6

# process noise Q
Q = dv * 100

# measurement variance R
R = 2 / Q

# initial system state = (x, P)
system = Gaussian(signal[0], 0)

# process = (u, Q)
process = Gaussian(0, Q)

# kalaman filter
kf = KalmanFilter().filter(signal, R, system, process)

In [ ]:
velocity = signal.exponential(0.275)

In [ ]:
kf_acceleration = kf.estimated.exponential(0.3).trend.div(3.6 * dt)

In [ ]:
slip = (velocity.level - kf.estimated <= -0.5)

In [ ]:
slip_flag = slip.enter_positive()

In [ ]:
go.Figure([
    Trace.from_dict('$VREF', df).plot(),
    Trace.from_dict('$VAG1', df).plot(),
    Trace.from_dict('$VAG2', df).plot(),
    signal.plot(),
    velocity.level.plot(),
    kf.estimated.plot(),
    kf.estimated.exponential(0.3).trend.div(3.6 * dt).plot(),
    velocity.trend.div(3.6 * dt).plot(),
    slip_flag.digital.plot(),
]).update_layout(
    title='Vehicle: Velocity',
    height=600,        
    margin=dict(t=50, b=60, l=50, r=50),
    xaxis_title='samples',
    yaxis_title='Velocity [km/h]',
)

## Signal Analysis: Brake Actuator

### Brake Actuator: Brake Pressure Exponential Smoothing

In [ ]:
original = Trace.from_dict('$PBZI', df)

In [ ]:
pressure = original.exponential(alpha=0.2)

In [ ]:
# smoothing factor slider
factor = widgets.FloatSlider(
    value=0.175, 
    min=0.0, 
    max=1.0, 
    step=0.005, 
    readout_format='.3f',
    description="alpha",
    layout=widgets.Layout(width='80%'),
    continuous_update=True)

# samples range slider
samples = widgets.IntRangeSlider(
    value=[355, 1300],
    min=0, 
    max=len(rows), 
    step=5, 
    description="samples",
    layout=widgets.Layout(width='80%'),
    continuous_update=True)

# figure widget
fw = go.FigureWidget([
    original.plot(),
    pressure.level.relabel('Pressure [bar]').plot(),
    original.sub(pressure.level).relabel('Error pressure [bar]').plot(fill='tozeroy')
]).update_layout(
    title='Brake Actuator: Pressure',
    height=600,
    margin=dict(t=50, b=60, l=50, r=50),
    xaxis_title='samples',
    yaxis_title='Pressure [bar]'
).update_traces(
    mode='lines'
)

# callback
def response(change):
    """ Callback to update the figure widget. """
    _smoothed = original.exponential(factor.value)
    _error = original - _smoothed.level
    
    with fw.batch_update():
        fw.data[0].y = original.samples[samples.value[0]:samples.value[1]]
        fw.data[1].y = _smoothed.level.samples[samples.value[0]:samples.value[1]]
        fw.data[2].y = _error.samples[samples.value[0]:samples.value[1]]

# observers        
factor.observe(response, names="value")
samples.observe(response, names="value")

# update figure widget with callback
response(dict())

# create widgets
widgets.VBox([factor, samples, fw])

### Brake Actuator: Air-Consumpiton

In [ ]:
make_subplots(
    rows=2, 
    cols=1,
    shared_xaxes=True,
    row_heights = (1, 1),
    vertical_spacing=0.005,
    x_title='samples'
).add_trace(
    pressure.level.delta().max(0).accumulate().div(dt).relabel('Brake-Reservoir').plot(),
    row=2, col=1
).update_yaxes(
    title = 'Air-consumption [Nl]',
    row=2, col=1
).add_trace(
    pressure.level.delta().min(0).accumulate().div(-dt).relabel('Slide-Protection').plot(),
    row=2, col=1
).add_trace(
    pressure.level.plot(), 
    row=1, col=1,
).update_yaxes(
    title = 'Pressure [bar]',
    row=1, col=1
).update_layout(
    title='Brake Actuator: Air-Consumption',
    height=800,
    margin=dict(t=50, b=60, l=50, r=50)
).update_traces(
    fill='tozeroy'
)

### Brake Actuator: Deceleration (based on Brake Actuatur Pressure)

Brake actuator force:

$$F_{actuator} = \frac{(P_{actuator} \cdot i_{actuator} - F_{spring} ) \cdot \mu_{pad} \cdot r_{disc}}{r_{wheel}}$$

Application pressure of the brake actuator in $bar$:

In [ ]:
p0 = 0.356535

Acceleration-pressure ratio of the brake actuator in $\frac{m}{s^2 \cdot bar}$:

In [ ]:
dadP = -0.47775

Applied brake actuator deceleration in $\frac{m}{s^2}$:

In [ ]:
deceleration = Trace('Deceleration [m/s^2]', (pressure.level - p0).max(0).mul(dadP))

## Signal Analysis: Wheel Movement

### Wheel Speed Signal: Name 

In [ ]:
signal = '$VAG1'

### Wheel Speed: Exponential Smoothing

In [ ]:
speed = Trace.from_dict(f'{signal}', df).exponential(alpha=0.275)

In [ ]:
go.Figure([
    Trace.from_dict(f'{signal}', df).plot(),
    speed.level.plot(),
    Trace.from_dict(f'{signal}', df).sub(speed.level).relabel('Error').plot(fill='tozeroy')
]).update_layout(
    title='Wheel: Speed',
    height=600,        
    margin=dict(t=50, b=60, l=50, r=50),
    xaxis_title='samples',
    yaxis_title='Speed [km/h]',
)

### Wheel Acceleration: Exponential Smoothing

In [ ]:
acceleration = Trace(f'{signal}:acceleration', speed.trend / (3.6 * dt)).exponential(alpha=0.275)

In [ ]:
go.Figure([
    speed.trend.div(3.6 * dt).plot(),
    acceleration.level.plot(),
    speed.trend.div(3.6 * dt).sub(acceleration.level).relabel('Error').plot(fill='tozeroy')
]).update_layout(
    title='Wheel: Acceleration',
    height=600,        
    margin=dict(t=50, b=60, l=50, r=50),
    xaxis_title='samples',
    yaxis_title='Acceleration [m/s^2]',
)

### Wheel Jerk: Exponential Smoothing

In [ ]:
jerk = Trace(f'{signal}:jerk', acceleration.trend / dt).exponential(alpha=0.275)

In [ ]:
go.Figure([
    acceleration.trend.div(dt).plot(),
    jerk.level.plot(),
    acceleration.trend.div(dt).sub(jerk.level).relabel('Error').plot(fill='tozeroy')
]).update_layout(
    title='Wheel: Jerk',
    height=600,        
    margin=dict(t=50, b=60, l=50, r=50),
    xaxis_title='samples',
    yaxis_title='Jerk [m/s^3]',
)

### Wheel Snap: Exponential Smoothing

In [ ]:
snap = Trace(f'{signal}:snap', jerk.trend / dt).exponential(alpha=0.3)

In [ ]:
go.Figure([
    jerk.trend.div(dt).plot(),
    snap.level.plot(),
    jerk.trend.div(dt).sub(snap.level).relabel('Error').plot(fill='tozeroy')
]).update_layout(
    title='Wheel: Snap',
    height=600,        
    margin=dict(t=50, b=60, l=50, r=50),
    xaxis_title='samples',
    yaxis_title='Snap [m/s^4]',
)

### Wheel: Adhesion Coefficient

Wheel adhesion coefficient:

$$\mu_{wheel} = \frac{\frac{a_{wheel} \cdot J_{axis}}{r_{wheel}^2} - F_{actuator}}{m_{axis} \cdot g}$$

Gravitational constant in $\frac{m}{s^2}$

In [ ]:
g = 9.81

Rotational mass **ratio** of the wheel axis:

In [ ]:
m_rot = 0.11735

Wheel adhesion coefficient

In [ ]:
coefficient = Trace('Adhesion', (m_rot * acceleration.level - dadP * (pressure.level - p0).max(0)) / g)

In [ ]:
adhesion = coefficient.exponential(0.4)

In [ ]:
go.Figure([
    coefficient.plot(),
    adhesion.level.plot(),
    adhesion.trend.mul(10).plot(fill='tozeroy'),
    adhesion.trend.left_positive().mul(0.1).relabel('Adhesion Maximum').digital.plot(),
    slip_flag.mul(0.15).relabel('Slip Flag').digital.plot(),
]).update_layout(
    title='Wheel: Adhesion Coefficient',
    height=600,        
    margin=dict(t=50, b=60, l=50, r=50),
    xaxis_title='samples',
    yaxis_title='Adhesion coefficient [1]',
)

### Wheel: Density of the Pressure and Adhesion Coefficient

In [ ]:
# samples range slider
samples = widgets.IntRangeSlider(
    value=[360, 1300],
    min=0, 
    max=len(rows), 
    step=5, 
    description="samples",
    layout=widgets.Layout(width='80%'),
    continuous_update=True)

# figure widget
fw = go.FigureWidget([
    go.Histogram2dContour(
        name='Density',
        x = pressure.level.samples,
        y = adhesion.level.samples,
        colorscale = 'Blues',
        reversescale = False,
        xaxis = 'x',
        yaxis = 'y',
        xbins=dict(start=-0.1, end=3.3, size=0.1),
        ybins=dict(start=-0.005, end=0.15, size=0.005),
        contours = dict(
            showlabels=True
        ),
        histnorm ='percent',
        colorbar=dict(
            title='Density [%]',
            titleside='right',
            len=0.8
        )
    ),
    go.Scatter(
        name='Points',
        x = pressure.level.samples,
        y = adhesion.level.samples,
        xaxis = 'x',
        yaxis = 'y',
        mode = 'markers',
        marker = dict(color='rgba(0,0,0,0.3)', size=4)
    ),
    go.Histogram(
        name='Pressure',
        x = pressure.level.samples,
        yaxis = 'y2',
        histnorm ='percent',
        xbins=dict(start=-0.1, end=3.3, size=0.1),
        marker = dict(color = 'rgba(0,0,0,1)'),
    ),
    go.Histogram(
        name='Adhesion',
        y = adhesion.level.samples,
        xaxis = 'x2',
        histnorm ='percent',
        ybins=dict(start=-0.005, end=0.15, size=0.005),
        marker = dict(color='rgba(0,0,0,1)'),
    ),
]).update_layout(
    xaxis = dict(
        title='Pressure [bar]',
        zeroline = False,
        domain = [0,0.85],
        showgrid = False
    ),
    yaxis = dict(
        title='Adhesion coefficient [1]',
        zeroline = False,
        domain = [0,0.85],
        showgrid = False
    ),
    xaxis2 = dict(
        zeroline = False,
        domain = [0.865,1],
        showgrid = False
    ),
    yaxis2 = dict(
        zeroline = False,
        domain = [0.865,1],
        showgrid = False
    ),
    title='Wheel: Density of the Pressure and Adhesion Coefficient',
    margin=dict(t=50, b=50, l=50, r=50),
    height = 800,
    legend_orientation="h",
    bargap = 0.1,
    hovermode = 'closest',
    showlegend = True
)

# callback
def response(change):
    """ Callback to update the figure widget. """
    
    # re-create quiver figure
    _contour = go.Histogram2dContour(
        x = pressure.level.samples[samples.value[0]:samples.value[1]],
        y = adhesion.level.samples[samples.value[0]:samples.value[1]])
    
    with fw.batch_update():
        fw.data[0].z = _contour.z
        fw.data[0].x = _contour.x
        fw.data[0].y = _contour.y
        fw.data[1].x = pressure.level.samples[samples.value[0]:samples.value[1]]
        fw.data[1].y = adhesion.level.samples[samples.value[0]:samples.value[1]]
        fw.data[1].text = [f'index: {i}' for i in range(samples.value[0], samples.value[1])]
        fw.data[2].x = pressure.level.samples[samples.value[0]:samples.value[1]]
        fw.data[3].y = adhesion.level.samples[samples.value[0]:samples.value[1]]
        
# observers
samples.observe(response, names="value")

# update figure widget with callback
response(dict())

# create widgets
widgets.VBox([fw, samples])

### Wheel: Density of the Pressure and Adhesion Coefficient

In [ ]:
def density(x: Trace, 
            y: Trace, 
            xgrid: Union[int, Dict], 
            ygrid: Union[int, Dict], 
            start: Optional[int] = None, 
            end: Optional[int] = None,
            **kwargs) -> Dict:
    """ Returns the density in percent computed from the x,y-sample pairs of 
    the *x* and *y* trace.
    
    param x: x-axis trace
    param y: y-axis trace
    param xgrid: integer number of bins of the x-axis grid or 
        a dictornary with the x-axis 'start', 'end' and 'tick' value.
    param ygrid: integer number of bins of the y-axis grid or 
        a dictornary with the y-axis 'start', 'end' and 'tick' value.
    param start: optional start index of the samples in the traces.
    param end: optional end index of the samples in the traces.
    """
    # slice x,y-samples pairs
    pairs = list(zip(x, y))
    
    if pairs:
        # slice pairs
        start = 0 if start is None else max(0, start)
        if end is None:
            pairs = pairs[start:]
        else:
            pairs = pairs[start:end]
            
    if not pairs:
        return dict(x=list(), y=list(), z=list())
    
    # unpack sliced pairs       
    x, y = zip(*pairs)
    
    # x-axis   
    xmax = max(x)
    xmin = min(x)

    if isinstance(xgrid, int):
        xrange = (xmax - xmin)
        if not xrange:
            xbins=1
        else:
            xbins=max(xgrid, 1)
        xtick = xrange / xbins
        if xtick <= 0.0:
            xtick = 1.0
    else:
        xmax = xgrid.get('end', xmax)
        xmin = xgrid.get('start', xmin)
        xrange = max(xmax - xmin, 0)
        xtick = xgrid.get('tick', 1.0) 
        if xtick <= 0.0:
            xtick = 1.0
        xbins, _ = divmod(xrange, xtick)
        xbins=max(int(xbins), 1)
              
    # y-axis
    ymax = max(y)
    ymin = min(y)

    if isinstance(ygrid, int):
        yrange = (ymax - ymin)
        if not yrange:
            ybins=1
        else:
            ybins=max(ygrid, 1)
        ytick = yrange / ybins
        if ytick <= 0.0:
            ytick = 1.0
    else:
        ymax = ygrid.get('end', ymax)
        ymin = ygrid.get('start', ymin)
        yrange = max(ymax - ymin, 0)
        ytick = ygrid.get('tick', 1.0)
        if ytick <= 0.0:
            ytick = 1.0
        ybins, _ = divmod(yrange, ytick)           
        ybins=int(max(ybins, 1))
      
    # shape of the grid
    shape = [
        xbins + 1 if xrange else xbins,
        ybins + 1 if yrange else ybins,
    ]
    
    # x-axis values of the grid
    xvalues = [xmin + xtick * i for i in range(shape[0])]
    
    # y-axis values of the grid
    yvalues = [ymin + ytick * i for i in range(shape[1])]
    
    # zeroed z-axis density of the x,y-samples
    zvalues = [[0.0] * (shape[0]) for _ in range(shape[1])]

    # z-axis percent of a x,y-sample
    ztick = 100.0 / max(len(x), 1)

    for _y, _x in zip(y, x):
        # y-index of z-axis nested list
        yindex = math.floor((_y - ymin) / ytick)
        if yindex < 0 or yindex >= shape[1]:
            continue
        # x-index of z-axis nested list
        xindex = math.floor((_x - xmin) / xtick)
        if xindex < 0 or xindex >= shape[0]:
            continue
        # z-axis values
        zvalues[yindex][xindex] += ztick

    return dict(x=xvalues, y=yvalues, z=zvalues)

In [ ]:
# samples range slider
samples = widgets.IntRangeSlider(
    value=[360, 1300],
    min=0, 
    max=len(rows), 
    step=5, 
    description="samples",
    layout=widgets.Layout(width='80%'),
    continuous_update=True)

# figure widget
fw = go.FigureWidget([
    go.Surface(
        name='Density',
        x=[],
        y=[],
        z=[],
        colorscale='Blues',
        reversescale =True,
        colorbar=dict(
            title='Density [%]',
            titleside='right',
            len=0.8
        )
    )
]).update_layout(
    title="Wheel: Pressure-Adhesion Density",
    height=900,
    margin=dict(t=50, b=50, l=50, r=50),
    scene_aspectratio=dict(x=1.5, y=1, z=0.5),
    scene_xaxis=dict(title='Pressure [bar]'),
    scene_yaxis=dict(title='Adhesion [1]'),
    scene_zaxis=dict(title='Density [%]'),
    # camera settings
    scene_camera=dict(
        eye=dict(x=-0.1, y=-1.25, z=1.25)),
    updatemenus=[
        dict(
            type = "buttons",
            showactive=True,
            xanchor="left",
            x=1,
            y=1,
            buttons=list([
                dict(
                    args=["type", "surface"],
                    label="3D Surface",
                    method="restyle"
                ),
                dict(
                    args=["type", "heatmap"],
                    label="Heatmap",
                    method="restyle"
                )
            ]),
        )
    ]
)

# callback
def response(change):
    """ Callback to update the figure widget. """
        
    # compute density data
    data = density(
        x=pressure.level,
        y=adhesion.level,
        xgrid=dict(start=-0.1, end=3.3, tick=0.1),
        ygrid=dict(start=-0.01, end=0.15, tick=0.005),
        start=samples.value[0],
        end=samples.value[1]
    )
    
    with fw.batch_update():
        fw.data[0].x = data['x']
        fw.data[0].y = data['y']
        fw.data[0].z = data['z']
        
# observers
samples.observe(response, names="value")

# update figure widget with callback
response(dict())

# create widgets
widgets.VBox([fw, samples])

### Wheel: Acceleration-Jerk State-Quadrants

In [ ]:
@dataclass
class Square:
    """ Square. """
    #: Square label
    label: str
    #: Square x-values
    x: List
    #: Square y-values
    y: List
    #: Square color
    color: str = 'DodgerBlue'
    
    def plot(self, **kwargs):
        """ Scatter 2D plot. """
        settings = dict(
            name=self.label,
            x=self.x,
            y=self.y,
            mode='markers',
            marker_color=self.color,
            fill='toself')
        settings.update(**kwargs)
        return go.Scatter(**settings)

### Wheel: Acceleration-Jerk State Node

#### Wheel: Acceleration-Jerk State Nodes

In [ ]:
nodes= [
    State2D(name='Q1', x=-1, y=-1, color='IndianRed'),
    State2D(name='Q1 > Q2', x=-1, y=0, color='Orange'), 
    State2D(name='Q1 < Q2', x=-0.75, y=0, color='IndianRed'), 
    State2D(name='Q2', x=-1, y=1, color='Orange'),
    State2D(name='Q2 < Q3', x=0, y=0.75, color='Orange'), 
    State2D(name='Q1 < Q4', x=0, y=-1, color='IndianRed'),
    State2D(name='Q0', x=0, y=0), 
    State2D(name='Q2 > Q3', x=0, y=1, color='DodgerBlue'),
    State2D(name='Q1 > Q4', x=0, y=-0.75, color='LightSeaGreen'),
    State2D(name='Q4', x=1, y=-1, color='LightSeaGreen'), 
    State2D(name='Q3 < Q4', x=0.75, y=0, color='DodgerBlue'),
    State2D(name='Q3 > Q4', x=1, y=0, color='LightSeaGreen'),
    State2D(name='Q3', x=1, y=1, color='DodgerBlue'),
]

#### Wheel: Acceleration-Jerk State Machine

In [ ]:
machine = Statemachine.create(nodes, -6)

#### Wheel: Acceleration-Jerk States

In [ ]:
def state2d(a: Trace, r: Trace, b0: Union[Trace, float] = 0.0) -> Trace:
    """ Returns a trace with the 2-dimensional node states computed 
    from the traces.
    
    :param a: wheel acceleration trace
    :param r: wheel jerk trace
    :param b0: translational deceleration
    """
    a0 = a - b0
    
    a_sign = a0.sign()
    a_enter_negative = a_sign.enter_negative()
    a_left_negative = a_sign.left_negative()
    
    r_sign = r.sign()
    r_left_positive = r_sign.left_positive()
    r_enter_positive = r_sign.enter_positive()
    
    samples = list()

    for i in range(len(a)):
        if a_enter_negative[i]:
            if r_sign[i] <= 0:
                # Q4 -> Q1
                state = -1
            else:
                # Q3 -> Q2
                state = -2
        elif a_left_negative[i]:
            if r_sign[i] == 1:
                # Q2 -> Q3
                state = 1
            else:
                # Q1 -> Q4
                state = 2               
        elif r_enter_positive[i]:
            if a_sign[i] == -1:
                # Q1 -> Q2: -b_max
                state = -5
            else:
                # Q4 -> Q3: +a_min                
                state = 4
        elif r_left_positive[i]:
            if a_sign[i] >= 0:
                # Q3 -> Q4: +a_max
                state = 5
            else:
                # Q2 -> Q1: -b_min
                state = -4
        elif a_sign[i] == -1 and r_sign[i] == 1:
            # Q2
            state = -3
        elif a_sign[i] == -1 and r_sign[i] <= 0:
            # Q1
            state = -6
        elif a_sign[i] >= 0 and r_sign[i] <= 0:
            # Q4
            state = 3
        elif a_sign[i] >=0 and r_sign[i] == 1:
            # Q3
            state = 6
        else:
            # Q0
            state = 0
        samples.append(state)
            
    return Trace('State', samples)

### Wheel: Deceleration Estimator

In [ ]:
@dataclass(eq=True, order=True, frozen=True)
class WheelDecelerationEstimator:
    """ Deceleration estimator results."""
    #: brake actuator pressure [bar]
    pressure: float = 0.0
    #: translational velocity [km/h]
    velocity: float = 0.0
    #: wheel speed [km/h]
    speed: float = 0.0
    #: wheel accleration [m/s^2]
    acceleration: float = 0.0
    #: wheel jerk [m/s^3]
    jerk: float = 0.0
    #: wheel adhesion coefficient [1]
    adhesion: float = 0.0
    #: wheel slip [km/h]
    slip: float = 0.0
    #: relative wheel slip [%]
    relative_slip: float = 0.0
    #: translational deceleration [m/s^2]
    deceleration: float = 0.0
    #: increase translational deceleration (1) or not (0)
    increase: int = 0
    #: decrease translational deceleration (1) or not (0)
    decrease: int = 0
    #: acceleration-deceleration, jerk vector amplitude
    amplitude: float = 0.0
    #: acceleration-deceleration, jerk vector euclidean distance 
    distance: float = 0.0
    #: acceleration-deceleration, jerk vector dot product 
    dot: float = 0.0
    #: acceleration-deceleration, jerk vector phase [-180..+180]
    phase: float = 0.0
    #: wheel acceleration-deceleration-jerk state [1]
    state: int = 0

In [ ]:
@dataclass(eq=False)
class WheelDecelerationEstimatorTraces(Traces):
    """ Collection of :class:`WheelDecelerationEstimator` traces.
    """
    #: brake actuator pressure trace
    pressure: Trace = field(default_factory=Trace)
    #: translational velocity estimation trace
    velocity: Trace = field(default_factory=Trace)
    #: wheel speed trace
    speed: Trace = field(default_factory=Trace)
    #: wheel accleration trace
    acceleration: Trace = field(default_factory=Trace)
    #: wheel jerk trace
    jerk: Trace = field(default_factory=Trace)
    #: wheel adhesion coefficient trace
    adhesion: Trace = field(default_factory=Trace)
    #: wheel slip estimation trace
    slip: Trace = field(default_factory=Trace)
    #: relative wheel slip estimation trace 
    relative_slip: Trace = field(default_factory=Trace)
    #: translational deceleration estimation trace
    deceleration: Trace = field(default_factory=Trace)
    #: increase translational deceleration estimation trace
    increase: Trace = field(default_factory=Trace)
    #: decrease translational deceleration estimation trace
    decrease: Trace = field(default_factory=Trace)
    #: acceleration-deceleration, jerk vector amplitude trace
    amplitude: Trace = field(default_factory=Trace)
    #: acceleration-deceleration, jerk vector euclidean distance trace
    distance: Trace = field(default_factory=Trace)
    #: acceleration-deceleration, jerk vector dot product trace
    dot: Trace = field(default_factory=Trace)
    #: acceleration-deceleration, jerk vector phase trace
    phase: Trace = field(default_factory=Trace)
    #: wheel acceleration-deceleration-jerk state [1]
    state: Trace = field(default_factory=Trace)    
    @classmethod
    def from_traces(
        cls,
        dt: float,
        b0: float,
        pressure: ExponentialSmoothingTraces, 
        deceleration: Trace,
        speed: ExponentialSmoothingTraces, 
        acceleration: ExponentialSmoothingTraces, 
        jerk: ExponentialSmoothingTraces,
        adhesion: ExponentialSmoothingTraces,
        **kwargs) -> WheelDecelerationEstimatorTraces:
        """
        Estimates the wheel :attr:`slip` and the translational :attr:`deceleration`.
    
        :param dt: sampling time of the traces in seconds
        :param b0: initial estimation of the translational deceleration
        :param pressure: smoothed brake actuator pressure traces
        :param deceleration: brake actuator applied deceleration trace
        :param speed: smoothed wheel speed traces
        :param acceleration: smoothed wheel acceleration traces
        :param jerk: smoothed wheel jerk traces
        """
        # initial slip integrator state
        _slip = 0.0
    
        # initial translational deceleration estimator state
        _translational = [b0, b0]
        _decrease = 0
        _increase = 0
        
        # translational deceleration estimator: acceleration maximum look-up
        _r_enter_positive = jerk.level.enter_positive()
        _a_maximum = 0
        
        # local adhesion coefficient maximum
        _adhesion_max = adhesion.trend.left_positive().move(0)
        _adhesion_max = slip_flag
        _init = False
        
        samples = list()
        for i, _inputs in enumerate(zip(pressure.level, 
                                        deceleration, 
                                        speed.level, 
                                        speed.trend, 
                                        acceleration.level.window(2), 
                                        jerk.level.window(2),
                                        adhesion.level)):
            _pressure, _deceleration, _speed, _trend, _acceleration, _jerk, _adhesion = _inputs
                      
            if not _init and _pressure > 0 and pressure.trend[i] > 0 and _adhesion_max[i] == 1 and _acceleration[-1] < 0 and _jerk[-1] < -1.5:
                #_translational[-1] = max(_acceleration[-1], _translational[-1])
                _translational[-1] = max(kf_acceleration[i], _translational[-1])
                _init = True
            
            # wheel acceleration based on speed trend
            _trend /= (3.6 * dt)
            _trend = _acceleration[-1]
            
            # wheel slip integrator
            if _increase:
                _slip = min(_slip - (_trend - _translational[-1]) * 3.6 * dt, 0)
            else:
                _slip = min(_slip + (_trend - _translational[-1]) * 3.6 * dt, 0)
                        
            # compute translational velocity
            _velocity = abs(_slip) + _speed
        
            # wheel iterator states
            _s_sign = np.sign(_slip)
            _a_sign = np.sign(_acceleration[-1] - _translational[-1])
            _r_sign = np.sign(_jerk[-1])

            # increase translational deceleration
            if _a_sign > -1:
                _a_maximum = 0
                _increase  = 0
            elif _r_sign == -1 and _a_maximum == 1:
                _a_maximum = 0
                _increase = 1
            elif _r_sign == -1:
                _a_maximum = 0
            elif _r_enter_positive[i] == 1:
                _a_maximum = 1
                _increase = 0
                
            # decrease translational deceleration
            if _a_sign == 1 and _r_sign >= 0 and _s_sign == 0:
                _decrease = 1
            elif _a_sign == -1 and _r_sign <= 0 and _s_sign == 0:
                _decrease = 1
            else:
                _decrease = 0

            # translational deceleration threshold        
            if _increase:
                _translational.append(max(_translational[-1] + min(_deceleration * dt * 0.75, -0.005), -2.2))
            elif _decrease and _pressure > 0:
                _translational.append(min(_translational[-1] - min(_deceleration * dt * 0.5, -0.0025), -0.2))
            else:
                _translational.append(_translational[-1])
                
            _translational = _translational[-2:]
            
            # a-b,r-points of the a-b,r-vectors
            _points = [(a - b, r) for a, b, r in zip(_acceleration, _translational, _jerk)]
            _distance = math.dist(*_points)
            _dot = np.dot(*_points)
            _points = [Point2D(*point) for point in _points]
            
            # a-b,r-state
            _state = state2d(Trace('a', _acceleration), Trace('r', _jerk), Trace('b0', _translational))
            
            # results
            results = WheelDecelerationEstimator(
                pressure = _pressure,
                velocity = _velocity,
                speed=_speed,
                acceleration=_acceleration[-1],
                jerk=_jerk[-1],
                adhesion=_adhesion,
                slip=_slip,
                relative_slip = _slip / _velocity * 100.0 if _velocity != 0.0 else 0.0,
                deceleration=_translational[-1],
                increase=_increase,
                decrease=_decrease,
                amplitude=_points[-1].r,
                distance=_distance,
                dot=_dot,
                phase=_points[-1].phi * (180 / math.pi),
                state=_state.samples[-1])
     
            samples.append(asdict(results))
    
        traces = dict()
        label = kwargs.get('label', 'Estimator')
        for key, value in as_traces(samples).items():
            traces[key] = Trace(label=f'{label}:{key}',
                                samples=value)
        return cls(**traces)

In [ ]:
estimator = WheelDecelerationEstimatorTraces.from_traces(dt, measurement.a_mean * 2, pressure, deceleration, speed, acceleration, jerk, adhesion)

In [ ]:
make_subplots(
    rows=8, 
    cols=1,
    shared_xaxes=True,
    row_heights = (1, 0.25, 1, 0.75, 0.5, 1, 0.1, 0.1),
    specs=[
        [{"secondary_y": True}], 
        [{"secondary_y": True}],
        [{"secondary_y": True}],
        [{"secondary_y": True}],
        [{"secondary_y": True}], 
        [{"secondary_y": True}], 
        [{"secondary_y": True}], 
        [{"secondary_y": True}]
    ],
    vertical_spacing=0.015,
    x_title='samples'
).add_trace(
    estimator.velocity.relabel('Velocity [km/h]').plot(),
    row=1, col=1
).add_trace(
    # measurement.speed.relabel('Vehicle speed [km/h]').plot(),
    Trace.from_dict('$VREF', df).plot(),
    row=1, col=1,
).add_trace(
    estimator.pressure.relabel('Pressure [bar]').plot(fill='tozeroy'),
    row=1, col=1,
    secondary_y=True
).add_trace(
    estimator.speed.relabel('Speed [km/h]').plot(),
    row=1, col=1
).add_trace(
    estimator.relative_slip.relabel('Relative slip [%]').plot(),
    row=2, col=1,
    secondary_y=True
).add_trace(
    estimator.slip.relabel('Slip [km/h]').plot(fill='tozeroy'),
    row=2, col=1
).add_trace(
    estimator.phase.relabel('Phase [degree]').plot(fill='tozeroy'), 
    row=3, col=1,
    secondary_y=True
).add_trace(
    estimator.amplitude.relabel('Amplitude [1]').plot(), 
    row=3, col=1,
).add_trace(
    estimator.distance.relabel('Euclidan Distance').plot(), 
    row=3, col=1,
).add_trace(
    estimator.dot.mul(0.1).relabel('Dot Product').plot(), 
    row=3, col=1,
).update_yaxes(
    range=[-180, 180],
    tickvals=[-180, -135, -90, -45, 0, 45, 90, 135, 180],
    row=3, col=1,
    secondary_y=True
).add_trace(
   estimator.state.relabel('State').digital.plot(fill='tozeroy'), 
    row=4, col=1,
).update_yaxes(
    tickvals=machine.numbers,
    ticktext=machine.labels,
    row=4, col=1,
    secondary_y=False
).add_trace(
    estimator.adhesion.relabel('Adhesion [1]').plot(fill='tozeroy'), 
    row=5, col=1,
).add_trace(
    deceleration.relabel('Deceleration [m/s^2]').plot(), 
    row=6, col=1,
).add_trace(
    estimator.deceleration.relabel('Translational Deceleration [m/s^2]').plot(),
    row=6, col=1
).add_trace(
    estimator.acceleration.relabel('Acceleration [m/s^2]').plot(),
    row=6, col=1
).add_trace(
    Trace.from_dict('$VREF', df).exponential(0.3).trend.div(3.6 * dt).relabel('$AREF').plot(),
    row=6, col=1
).add_trace(
    estimator.jerk.relabel('Jerk [m/s^3]').plot(fill='tozeroy'),
    row=6, col=1,
    secondary_y=True
).add_trace(
    estimator.increase.relabel('Increase threshold').digital.plot(fill='tozeroy'),
    row=7, col=1
).add_trace(
    estimator.decrease.relabel('Decrease threshold').digital.plot(fill='tozeroy'),
    row=8, col=1
).update_layout(
    title='Translational Deceleration Estimator',
    height=1440,
    margin=dict(t=50, b=60, l=50, r=50),
).update_traces(
    mode='lines'
)

### Wheel: Acceleration-Jerk Vector 

In [ ]:
vector = VectorTraces(x=acceleration.level - estimator.deceleration, y=jerk.level)

### Wheel: Amplitude (based on Wheel Acceleration-Jerk Vector)

In [ ]:
amplitude = vector.r

### Wheel: Phase (based on Wheel Acceleration-JerkVector)

In [ ]:
phase = vector.phi.mul(180 / math.pi)

### Wheel: Acceleration-Jerk Vector (Polar Plot)

Polar plot to display the acceleration, jerk vectors of the measured wheel speed in the polar coordinate system.

In [ ]:
# decleration selector
selector = widgets.Dropdown(
    options=[('Manual', 1), ('Estimator', 2)],
    value=1,
    description='limit:'
)

# translational deceleration limit slider
limit = widgets.FloatSlider(
    value=measurement.a_mean, 
    min=-2, 
    max=0, 
    step=0.01, 
    description="limit",
    layout=widgets.Layout(width='50%'),
    continuous_update=True)

# samples range slider
samples = widgets.IntRangeSlider(
    value=[355, 1300],
    min=0, 
    max=len(rows), 
    step=5, 
    description="samples",
    layout=widgets.Layout(width='80%'),
    continuous_update=True)

# figure widget
fw = go.FigureWidget([
    VectorTraces(x=acceleration.level - limit.value, y=jerk.level).plot(
        name='Acceleration-Jerk Vectors')
]).update_layout(
    height=800
).update_traces(
    showlegend=True
)

# callback
def response(change):
    """ Callback to update the figure widget. """
    if selector.value == 2:
        _limit = estimator.deceleration
        limit.disabled = True
    else:
        limit.disabled = False
        _limit = limit.value

    # re-compute vector
    _vector = VectorTraces(x=acceleration.level - _limit, y=jerk.level)

    with fw.batch_update():
        fw.data[0].r = _vector.r.samples[samples.value[0]:samples.value[1]]
        fw.data[0].theta = _vector.theta.samples[samples.value[0]:samples.value[1]]
        fw.data[0].text = [f'index: {i}' for i in  range(samples.value[0], samples.value[1])]

# observers
selector.observe(response, names="value")
limit.observe(response, names="value")
samples.observe(response, names="value")

# update figure widget with callback
response(dict())

# create widgets
widgets.VBox([selector, limit, samples, fw])

### Wheel: Acceleration-Jerk Vector Sector Distribution (Polar Bar Plot)

Polar bar plot to display the sectorwise length distribution of the acceleration, jerk vectors of the measured wheel speed in the polar coordinate system.

In [ ]:
# partitions
partitions = 10

# sector in degres
sector = 15

# decleration selector
selector = widgets.Dropdown(
    options=[('Manual', 1), ('Estimator', 2)],
    value=1,
    description='limit:'
)

# translational deceleration limit slider
limit = widgets.FloatSlider(
    value=measurement.a_mean, 
    min=-2, 
    max=0, 
    step=0.01, 
    description="limit",
    layout=widgets.Layout(width='50%'),
    continuous_update=True)

# samples range slider
samples = widgets.IntRangeSlider(
    value=[355, 1300],
    min=0, 
    max=len(rows), 
    step=5, 
    description="samples",
    layout=widgets.Layout(width='80%'),
    continuous_update=True)

# figure widget
fw = go.FigureWidget(
    [go.Barpolar(
        name=f'{i * partitions}-{(i + 1) * partitions} %', 
        r=[], 
        theta0=sector/2) for i in range(partitions + 1)]
).update_layout(
    title='Sectorwise Length Distirbution of the Acceleration-Jerk Vectors',
    height=800,
)

# callback
def response(change):
    """ Callback to update the figure widget. """
    if selector.value == 2:
        _limit = estimator.deceleration
        limit.disabled = True
    else:
        limit.disabled = False
        _limit = limit.value
    
    # re-compute vector
    _vector = VectorTraces(x=acceleration.level - _limit, y=jerk.level)
    # vector radius partitions
    _norm = math.floor((_vector.r / max(_vector.r[samples.value[0]:samples.value[1]])) * 100 / (100 / partitions)).samples[samples.value[0]:samples.value[1]]
    # vector angel sectors
    _angel = math.floor(_vector.theta / sector).samples[samples.value[0]:samples.value[1]]
    
    # sectors
    _sectors = [[0] * math.floor(360 / sector) for _ in range(partitions + 1)]
    _tick = 100 / len(_norm)
    for i, j in zip(_norm, _angel):
        _sectors[i][j] += 1
        
    with fw.batch_update():
        for i, _r in enumerate(_sectors):
            fw.data[i].r = _r

# observers
selector.observe(response, names="value")
limit.observe(response, names="value")
samples.observe(response, names="value")

# update figure widget with callback
response(dict())

# create widgets
widgets.VBox([selector, limit, samples, fw])

### Wheel: Acceration-Jerk Vector (Quiver Plot)

Quiver plot to display the velocity of the acceleration, jerk vectors of the measured wheel speed with visualized wheel movement quadrants.

In [ ]:
# translational deceleration limit slider
limit = widgets.FloatSlider(
    value=measurement.a_mean,
    min=-2,
    max=0,
    step=0.01,
    description="limit",
    layout=widgets.Layout(width='50%'),
    continuous_update=True)

# samples range slider
samples = widgets.IntRangeSlider(
    value=[355, 1300],
    min=0,
    max=len(rows),
    step=5,
    description="samples",
    layout=widgets.Layout(width='80%'),
    continuous_update=True)

# figure widget
fw = go.FigureWidget(
    ff.create_quiver(
        name="Acceleration-Jerk Vectors",
        x=acceleration.level,
        y=jerk.level,
        u=acceleration.level.delta(),
        v=jerk.level.delta(),
        scale=0.75,
        scaleratio = 0.25,
        arrow_scale=0.5,
        line_width=2).data[0]
).add_trace(
    Square("Quadrant1", 
           [0, -1, -1, 0],
           [-1, -1, 0, 0],
           'IndianRed').plot()
).add_trace( 
    Square("Quadrant2", 
           [-1, -1, 0, 0],
           [0, 1, 1, 0],
           'Orange').plot()
).add_trace( 
    Square("Quadrant3", 
           [0, 1, 1, 0],
           [1, 1, 0, 0],
           'DodgerBlue').plot()
).add_trace(
    Square("Quadrant4", 
           [1, 1, 0, 0],
           [0, -1, -1, 0],
           'LightSeaGreen').plot()
).update_layout(
    title="Wheel: Velocity of the Acceleration-Jerk Vectors",
    height=800,
    margin=dict(t=50, b=50, l=50, r=50),
    legend_orientation="v",
    xaxis=dict(title="Acceleration [m/s^2]"),
    yaxis=dict(title="Jerk [m/s^3]")
)

# callback
def response(change):
    """ Callback to update the figure widget. """
    _acceleration = acceleration.level[samples.value[0]:samples.value[1]]
    _jerk = jerk.level[samples.value[0]:samples.value[1]]

    # extremas
    x_max = max(_acceleration)
    x_min = min(_acceleration)
    y_max = max(_jerk)
    y_min = min(_jerk)
    
    # compute indexes of the arrow points
    indexes = range(samples.value[0], samples.value[1])
    indexes = chain(
        # indexes for arrow stem points
        chain.from_iterable(map(lambda x: repeat(x, 3), indexes)),
        # indexes for arrow tip points
        chain.from_iterable(map(lambda x: repeat(x, 4), indexes))) 
    
    # re-create quiver figure
    quiver = ff.create_quiver(
                x=_acceleration, 
                y=_jerk, 
                u=acceleration.level.delta()[samples.value[0]:samples.value[1]], 
                v=jerk.level.delta()[samples.value[0]:samples.value[1]],
                scale=0.75,
                scaleratio = 0.25,
                arrow_scale=0.5,
                text=[f'index: {i}' for i in indexes],
                line_width=2)

    with fw.batch_update():
        # update square
        fw.data[1].x = [limit.value, x_min, x_min, limit.value]
        fw.data[1].y = [y_min, y_min, 0, 0]
        # update square
        fw.data[2].x = [x_min, x_min, limit.value, limit.value]
        fw.data[2].y = [0, y_max, y_max, 0]
        # update square
        fw.data[3].x = [limit.value, x_max, x_max, limit.value]
        fw.data[3].y = [y_max, y_max, 0, 0]
        # update square
        fw.data[4].x = [x_max, x_max, limit.value, limit.value]
        fw.data[4].y = [0, y_min, y_min, 0]
    
    # update quiver plot
    fw.data[0].update(quiver.data[0])

# observers    
limit.observe(response, names="value")
samples.observe(response, names="value")

# update figure widget with callback
response(dict())

# create widgets
widgets.VBox([limit, samples, fw])

### Wheel: Acceleration-Jerk States

In [ ]:
# decleration selector
selector = widgets.Dropdown(
    options=[('Manuel', 1), ('Estimator', 2)],
    value=1,
    description='limit:'
)

# translational deceleration limit slider
limit = widgets.FloatSlider(
    value=measurement.a_mean, 
    min=-2, 
    max=0, 
    step=0.01, 
    description="limit",
    layout=widgets.Layout(width='50%'),
    continuous_update=True)

# samples range slider
samples = widgets.IntRangeSlider(
    value=[355, 1300],
    min=0,
    max=len(rows),
    step=5,
    description="samples",
    layout=widgets.Layout(width='80%'),
    continuous_update=True)

# figure widget
fw = go.FigureWidget([
    go.Scatter(name='Quadrant1', fill='tozeroy', 
               line=dict(color='IndianRed', shape='hv')),
    go.Scatter(name='Quadrant2', fill='tozeroy',
               line=dict(color='Orange', shape='hv')),
    go.Scatter(name='Quadrant3', fill='tozeroy',
               line=dict(color='DodgerBlue', shape='hv')),
    go.Scatter(name='Quadrant4', fill='tozeroy',
               line=dict(color='LightSeaGreen', shape='hv')),
    state2d(acceleration.level, jerk.level, limit.value).digital.plot(line_color='#636EFA'),
    acceleration.level.sub(estimator.deceleration).relabel('Acclereation').plot(line_color='#EF553B'),
    estimator.deceleration.relabel('Declereation').plot(line_color='#00CC96'),
    jerk.level.relabel('Jerk').plot(line_color='#AB63FA', fill='tozeroy'),
]).update_layout(
    title="Wheel: Acceleration-Jerk States",
    height=800,
    margin=dict(t=50, b=50, l=50, r=50),
    legend_orientation="h",
    xaxis_title='samples',
    yaxis_title='State'
).update_traces(
   mode='lines'
)

# callback
def response(change):
    """ Callback to update the figure widget. """
    if selector.value == 2:
        _limit = estimator.deceleration
        limit.disabled = True
    else:
        limit.disabled = False
        _limit = [limit.value] * len(estimator.deceleration)
        
    _states = state2d(acceleration.level, jerk.level, _limit).samples[samples.value[0]:samples.value[1]]
    _q1 = [state if state in (-6, -4, -1) else 0 for state in _states]
    _q2 = [state if state in (-5, -3, -2) else 0 for state in _states]
    _q3 = [state if state in (6, 4, 1) else 0 for state in _states]
    _q4 = [state if state in (5, 3, 2) else 0 for state in _states]
        
    with fw.batch_update():
        fw.data[0].y = _q1
        fw.data[1].y = _q2
        fw.data[2].y = _q3
        fw.data[3].y = _q4
        fw.data[4].y = _states
        fw.data[5].y = acceleration.level.sub(_limit).samples[samples.value[0]:samples.value[1]]
        fw.data[6].y = _limit[samples.value[0]:samples.value[1]]
        fw.data[7].y = jerk.level.mul(0.1).samples[samples.value[0]:samples.value[1]]

# observers
selector.observe(response, names="value")
limit.observe(response, names="value")
samples.observe(response, names="value")

# update figure widget with callback
response(dict())

# create widgets
widgets.VBox([selector, fw, samples, limit])

### Wheel Acclerations-Jerk State-Transitions Table

In [ ]:
machine = machine.evaluate(state2d(acceleration.level, jerk.level, estimator.deceleration))

In [ ]:
machine.heatmap().update_layout(
    height=600,
    width=600,
)

In [ ]:
machine.table()

### Wheel: Acceleration-Jerk State-Transitions

In [ ]:
# decleration selector
selector = widgets.Dropdown(
    options=[('Manuel', 1), ('Estimator', 2)],
    value=1,
    description='limit:'
)

# translational deceleration limit slider
limit = widgets.FloatSlider(
    value=measurement.a_mean, 
    min=-2, 
    max=0, 
    step=0.01, 
    description="limit",
    layout=widgets.Layout(width='50%'),
    continuous_update=True)

# samples range slider
samples = widgets.IntRangeSlider(
    value=[355, 1300],
    min=0,
    max=len(rows),
    step=1,
    description="samples",
    layout=widgets.Layout(width='80%'),
    continuous_update=True)

# figure widget
fw = go.FigureWidget([
    Square("Quadrant1", 
           [0, -1, -1, 0],
           [-1, -1, 0, 0],
           'IndianRed').plot(),
    Square("Quadrant2", 
           [-1, -1, 0, 0],
           [0, 1, 1, 0],
           'Orange').plot(),
    Square("Quadrant3", 
           [0, 1, 1, 0],
           [1, 1, 0, 0],
           'DodgerBlue').plot(),
    Square("Quadrant4", 
           [1, 1, 0, 0],
           [0, -1, -1, 0],
           'LightSeaGreen').plot(),
    go.Scatter(
        name='Transitions',
        y=[],
        x=[],
        mode='lines',
        line_color='SlateGrey'
    ),
    go.Scatter(
        name='Last Transition',
        y=[],
        x=[],
        mode='lines',
        line_width=5,
        line_color='SlateGrey',       
    ),
    go.Scatter(
        name='Current State',
        y=[],
        x=[],
        mode='markers',
        marker_size=20,
        marker_color='SlateGrey',       
    ),
    go.Scatter(
        name='Wheel States',
        y=[node.y for node in machine.values()], 
        x=[node.x for node in machine.values()],
        marker_color=[node.color for node in machine.values()],
        mode='markers+text',
        marker_size=15,
        text=machine.labels,
        textposition=[
            'bottom left', 
            'top left', 
            'bottom center', 
            'top left', 
            'bottom center',
            'bottom center',
            'middle right',
            'top center',
            'top center',
            'bottom right',
            'top center',
            'bottom right',
            'top right'
        ]
    )
]).update_layout(
    title="Wheel: Acceleration-Jerk Transition-States",
    margin=dict(t=50, b=50, l=50, r=50),
    height=700,
    width=800,
    xaxis_title='Acceleration',
    xaxis_range=[-1.5, 1.5],
    yaxis_title='Jerk',
    yaxis_range=[-1.5, 1.5],    
)

# callback
def response(change):
    """ Callback to update the figure widget. """
    if selector.value == 2:
        _limit = estimator.deceleration
        limit.disabled = True
    else:
        limit.disabled = False
        _limit = limit.value
    
    _states = state2d(acceleration.level, jerk.level, _limit)
    _deltas = _states.delta()

    _states = _states[samples.value[0]:samples.value[1]]
    _deltas = _deltas[samples.value[0]:samples.value[1]]
    
    _timer = dict((i, 0) for i in machine.numbers)
    _transition = dict(x=[], y=[])
    
    for state, delta in zip(_states, _deltas):
        if delta:
            _timer[state] = 1.0
            _node = machine[state]
            _transition['x'].append(_node.x)
            _transition['y'].append(_node.y)
        else:
            _timer[state] += 1.0
        _timer.update(dict([(key, max(value - 0.1, 0)) for key, value in _timer.items() if key != state]))
        
    _states = [machine[state] for state in _states]
    _y = [node.y for node in _states]
    _x = [node.x for node in _states]

    with fw.batch_update():
        fw.data[4].y = _y
        fw.data[4].x = _x
        fw.data[5].y = _transition['y'][-2:]
        fw.data[5].x = _transition['x'][-2:]
        fw.data[6].y = [_y[-1]]
        fw.data[6].x = [_x[-1]]
        fw.data[7].text = [f'{node.name}: {time:.1f}' for node, time in zip(machine.values(), _timer.values())]
        
# observers
selector.observe(response, names="value")
limit.observe(response, names="value")
samples.observe(response, names="value")

# update figure widget with callback
response(dict())

# create widgets
widgets.VBox([selector, fw, samples, limit])

### Wheel: Acceration-Jerk-Snap Vector-Space

3D-Cone plot to display the acceleration, jerk, snap vectors of the measured wheel speed with visualized wheel movement cubes.

In [ ]:
@dataclass
class Cube:
    """ Cube. """
    #: Cube label
    label: str
    #: Cube x-values
    x: List
    #: Cube y-values
    y: List
    #: Cube z-values
    z: List
    #: Cube color
    color: str = 'blue'
    
    def plot(self, **kwargs):
        """ Mesh 3D plot. """
        settings = dict(
            name=self.label,
            x=self.x,
            y=self.y,
            z=self.z,
            i = [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
            j = [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
            k = [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
            opacity=0.1,
            color=self.color
        )
        settings.update(**kwargs)
        return go.Mesh3d(**settings)

In [ ]:
# translational deceleration limit slider
limit = widgets.FloatSlider(
    value=measurement.a_mean, 
    min=-2, 
    max=0, 
    step=0.01, 
    description="limit",
    layout=widgets.Layout(width='50%'),
    continuous_update=True)

# samples range slider
samples = widgets.IntRangeSlider(
    value=[355, 1300],
    min=0, 
    max=len(rows), 
    step=5, 
    description="samples",
    layout=widgets.Layout(width='80%'),
    continuous_update=True)

# figure widget
fw = go.FigureWidget([
    go.Cone(
        name='Acceleration-Jerk-Snap Vectors',
        x=acceleration.level.samples,
        y=jerk.level.samples,
        z=snap.level.samples,
        u=acceleration.level.delta().samples,
        v=jerk.level.delta().samples,
        w=snap.level.delta().samples,
        sizemode="scaled",
        sizeref=2,
        anchor="tip"),
    Cube('Quadrant1-1', 
         [0, -1, -1, 0, 0, -1, -1, 0], 
         [0, 0, -1, -1, 0, 0, -1, -1], 
         [0, 0, 0, 0, -1, -1, -1, -1],
         '#CD5C5C').plot(),
    Cube('Quadrant1+1', 
         [0, -1, -1, 0, 0, -1, -1, 0], 
         [0, 0, -1, -1, 0, 0, -1, -1],
         [0, 0, 0, 0, 1, 1, 1, 1],
         '#CD5C5C').plot(),
    Cube('Quadrant2+1', 
         [0, -1, -1, 0, 0, -1, -1, 0],
         [0, 0, 1, 1, 0, 0, 1, 1],  
         [0, 0, 0, 0, 1, 1, 1, 1],
         '#FFA500').plot(),
    Cube('Quadrant2-1', 
         [0, -1, -1, 0, 0, -1, -1, 0], 
         [0, 0, 1, 1, 0, 0, 1, 1],  
         [0, 0, 0, 0, -1, -1, -1, -1],
         '#FFA500').plot(),
    Cube('Quadrant3+1',
         [0, 1, 1, 0, 0, 1, 1, 0], 
         [0, 0, 1, 1, 0, 0, 1, 1],
         [0, 0, 0, 0, 1, 1, 1, 1], 
         '#1E90FF').plot(),
    Cube('Quadrant3-1',
         [0, 1, 1, 0, 0, 1, 1, 0], 
         [0, 0, 1, 1, 0, 0, 1, 1], 
         [0, 0, 0, 0, -1, -1, -1, -1],
         '#1E90FF').plot(),
    Cube('Quadrant4-1',
         [0, 1, 1, 0, 0, 1, 1, 0], 
         [0, 0, -1, -1, 0, 0, -1, -1],
         [0, 0, 0, 0, -1, -1, -1, -1],
         '#20B2AA').plot(),
    Cube('Quadrant4+1',
         [0, 1, 1, 0, 0, 1, 1, 0],  
         [0, 0, -1, -1, 0, 0, -1, -1], 
         [0, 0, 0, 0, 1, 1, 1, 1], 
         '#20B2AA').plot(),
]).update_layout(
    title="Wheel: Acceleration-Jerk-Snap Vectors",
    height=900,
    margin=dict(t=50, b=50, l=50, r=50),
    legend_orientation="h",
    scene_aspectratio=dict(x=2, y=1, z=1),
    scene_xaxis=dict(title='Acceleration [m/s^2]'),
    scene_yaxis=dict(title='Jerk [m/s^3]'),
    scene_zaxis=dict(title='Snap [m/s^4]'),
    # camera settings
    scene_camera=dict(
        eye=dict(x=0.5, y=-2, z=0.75))
).update_traces(
    showlegend=True
)

#callback
def response(change):
    """ Callback to update the figure widget. """
    # update data to display
    data = dict(
        x=acceleration.level[samples.value[0]:samples.value[1]],
        y=jerk.level[samples.value[0]:samples.value[1]],
        z=snap.level[samples.value[0]:samples.value[1]],
        u=acceleration.level.delta()[samples.value[0]:samples.value[1]],
        v=jerk.level.delta()[samples.value[0]:samples.value[1]],
        w=snap.level.delta()[samples.value[0]:samples.value[1]]
    )

    # extremas
    x_max = max(data['x'])
    x_min = min(data['x'])
    y_max = max(data['y'])
    y_min = min(data['y'])
    z_max = max(data['z'])
    z_min = min(data['z'])

    with fw.batch_update():
        # Cone
        fw.data[0].x = data['x']
        fw.data[0].y = data['y']
        fw.data[0].z = data['z']
        fw.data[0].u = data['u']
        fw.data[0].v = data['v']
        fw.data[0].w = data['w']
        fw.data[0].text = [f'index: {i}' for i in range(samples.value[0], samples.value[1])]
        # Cube
        fw.data[1].x = [limit.value, x_min, x_min, limit.value, limit.value, x_min, x_min, limit.value]
        fw.data[1].y = [0, 0, y_min, y_min, 0, 0, y_min, y_min]
        fw.data[1].z = [0, 0, 0, 0, z_min, z_min, z_min, z_min]
        # Cube       
        fw.data[2].x = [limit.value, x_min, x_min, limit.value, limit.value, x_min, x_min, limit.value]
        fw.data[2].y = [0, 0, y_min, y_min, 0, 0, y_min, y_min]
        fw.data[2].z = [0, 0, 0, 0, z_max, z_max, z_max, z_max]
        # Cube              
        fw.data[3].x = [limit.value, x_min, x_min, limit.value, limit.value, x_min, x_min, limit.value]
        fw.data[3].y = [0, 0, y_max, y_max, 0, 0, y_max, y_max]
        fw.data[3].z = [0, 0, 0, 0, z_max, z_max, z_max, z_max]
        # Cube              
        fw.data[4].x = [limit.value, x_min, x_min, limit.value, limit.value, x_min, x_min, limit.value] 
        fw.data[4].y = [0, 0, y_max, y_max, 0, 0, y_max, y_max]
        fw.data[4].z = [0, 0, 0, 0, z_min, z_min, z_min, z_min]
        # Cube              
        fw.data[5].x = [limit.value, x_max, x_max, limit.value, limit.value, x_max, x_max, limit.value]
        fw.data[5].y = [0, 0, y_max, y_max, 0, 0, y_max, y_max]
        fw.data[5].z = [0, 0, 0, 0, z_max, z_max, z_max, z_max]
        # Cube              
        fw.data[6].x = [limit.value, x_max, x_max, limit.value, limit.value, x_max, x_max, limit.value]
        fw.data[6].y = [0, 0, y_max, y_max, 0, 0, y_max, y_max]
        fw.data[6].z = [0, 0, 0, 0, z_min, z_min, z_min, z_min]
        # Cube              
        fw.data[7].x = [limit.value, x_max, x_max, limit.value, limit.value, x_max, x_max, limit.value]
        fw.data[7].y = [0, 0, y_min, y_min, 0, 0, y_min, y_min]
        fw.data[7].z = [0, 0, 0, 0, z_min, z_min, z_min, z_min]
        # Cube              
        fw.data[8].x = [limit.value, x_max, x_max, limit.value, limit.value, x_max, x_max, limit.value]
        fw.data[8].y = [0, 0, y_min, y_min, 0, 0, y_min, y_min] 
        fw.data[8].z = [0, 0, 0, 0, z_max, z_max, z_max, z_max]
        
    fw.update_layout(
        scene_xaxis=dict(
            range=[math.floor(x_min - 0.5), math.ceil(x_max + 0.5)]),
        scene_yaxis=dict(
            range=[math.floor(y_min - 0.5), math.ceil(y_max + 0.5)]),
        scene_zaxis=dict(
            range=[math.floor(z_min - 0.5), math.ceil(z_max + 0.5)]))

# observers
limit.observe(response, names="value")
samples.observe(response, names="value")

# update figure widget with callback
response(dict())

# create widgets
widgets.VBox([fw, samples, limit])

### Wheel: Slip Integrator (based on Wheel Acceleration and Translational Deceleration Estimation)

In [ ]:
slip = Trace(f'{signal}:slip', speed.trend.sub(measurement.a_mean * 3.6 * dt).sums_negative())

### Wheel: Translational Velocity (based on Wheel Slip Integrator)

In [ ]:
velocity = speed.level - slip

### Wheel: Density of the Relative Slip and Adhesion Coefficient

In [ ]:
# slip selector range
selector = widgets.Dropdown(
    options=[('Approximation', 1), ('Estimator', 2)],
    value=1,
    description='slip:'
)

# samples range slider
samples = widgets.IntRangeSlider(
    value=[360, 1300],
    min=0, 
    max=len(rows), 
    step=5, 
    description="samples",
    layout=widgets.Layout(width='80%'),
    continuous_update=True)

# figure widget
fw = go.FigureWidget([
    go.Histogram2dContour(
        name='Density',
        x = estimator.relative_slip.samples,
        y = estimator.adhesion.samples,
        colorscale = 'Blues',
        reversescale = False,
        xaxis = 'x',
        yaxis = 'y',
        xbins=dict(start=-0.5, end=33, size=0.5),
        ybins=dict(start=-0.005, end=0.15, size=0.005),
        contours = dict(
            showlabels=True
        ),
        histnorm ='percent',
        colorbar=dict(
            title='Density [%]',
            titleside='right',
            len=0.8
        )
    ),
    go.Scatter(
        name='Points',
        x = pressure.level.samples,
        y = estimator.relative_slip.samples,
        xaxis = 'x',
        yaxis = 'y',
        mode = 'markers',
        marker = dict(color='rgba(0,0,0,0.3)', size=4)
    ),
    go.Histogram(
        name='Relative Slip',
        x = estimator.relative_slip.samples,
        yaxis = 'y2',
        histnorm ='percent',
        xbins=dict(start=-1, end=35, size=0.5),
        marker = dict(color = 'rgba(0,0,0,1)'),
    ),
    go.Histogram(
        name='Adhesion',
        y = estimator.adhesion.samples,
        xaxis = 'x2',
        histnorm ='percent',
        ybins=dict(start=-0.005, end=0.15, size=0.005),
        marker = dict(color='rgba(0,0,0,1)'),
    ),
]).update_layout(
    xaxis = dict(
        title='Relative Slip [%]',
        zeroline = False,
        domain = [0,0.85],
        showgrid = False
    ),
    yaxis = dict(
        title='Adhesion coefficient [1]',
        zeroline = False,
        domain = [0,0.85],
        showgrid = False
    ),
    xaxis2 = dict(
        zeroline = False,
        domain = [0.865,1],
        showgrid = False
    ),
    yaxis2 = dict(
        zeroline = False,
        domain = [0.865,1],
        showgrid = False
    ),
    title='Wheel: Density of the Relative Slip and Adhesion Coefficient',
    margin=dict(t=50, b=50, l=50, r=50),
    height = 800,
    legend_orientation="h",
    bargap = 0.1,
    hovermode = 'closest',
    showlegend = True
)

# callback
def response(change):
    """ Callback to update the figure widget. """
    if selector.value == 2:
        _slip = -estimator.relative_slip
    else:
        _slip = -slip / velocity * 100
    # re-create contour figure
    _contour = go.Histogram2dContour(
        x = _slip.samples[samples.value[0]:samples.value[1]],
        y = estimator.adhesion.samples[samples.value[0]:samples.value[1]])
    
    with fw.batch_update():
        fw.data[0].z = _contour.z
        fw.data[0].x = _contour.x
        fw.data[0].y = _contour.y
        fw.data[1].x = _slip.samples[samples.value[0]:samples.value[1]]
        fw.data[1].y = estimator.adhesion.samples[samples.value[0]:samples.value[1]]
        fw.data[1].text = [f'index: {i}' for i in range(samples.value[0], samples.value[1])]
        fw.data[2].x = _slip.samples[samples.value[0]:samples.value[1]]
        fw.data[3].y = estimator.adhesion.samples[samples.value[0]:samples.value[1]]
        
# observers
samples.observe(response, names="value")
selector.observe(response, names="value")

# update figure widget with callback
response(dict())

# create widgets
widgets.VBox([selector, fw, samples])

### Wheel: Density of the Relative Slip and Adhesion Coefficient

In [ ]:
# slip selector range
selector = widgets.Dropdown(
    options=[('Approximation', 1), ('Estimator', 2)],
    value=1,
    description='slip:',
)

# samples range slider
samples = widgets.IntRangeSlider(
    value=[360, 1300],
    min=0, 
    max=len(rows), 
    step=5, 
    description="samples",
    layout=widgets.Layout(width='80%'),
    continuous_update=True)

# figure widget
fw = go.FigureWidget([
    go.Surface(
        name='Density',
        x=[],
        y=[],
        z=[],
        colorscale='Blues',
        reversescale =True,
        colorbar=dict(
            title='Density [%]',
            titleside='right',
            len=0.8
        )
    )
]).update_layout(
    title="Wheel: Relative-Slip-Adhesion Density",
    height=1000,
    margin=dict(t=50, b=50, l=50, r=50),
    scene_aspectratio=dict(x=1.5, y=1, z=0.5),
    scene_xaxis=dict(title='Relative Slip [%]'),
    scene_yaxis=dict(title='Adhesion [1]'),
    scene_zaxis=dict(title='Density [%]'),
    # camera settings
    scene_camera=dict(
        eye=dict(x=-0.1, y=-1.25, z=1.25)),
    updatemenus=[
        dict(
            type = "buttons",
            showactive=True,
            xanchor="left",
            x=1,
            y=1,
            buttons=list([
                dict(
                    args=["type", "surface"],
                    label="3D Surface",
                    method="restyle"
                ),
                dict(
                    args=["type", "heatmap"],
                    label="Heatmap",
                    method="restyle"
                )
            ]),
        )
    ]
)

# callback
def response(change):
    """ Callback to update the figure widget. """
    if selector.value == 2:
        _slip = -estimator.relative_slip
    else:
        _slip = -slip / velocity * 100
        
    # compute density data
    data = density(
        x=_slip,
        y=estimator.adhesion,
        xgrid=dict(start=-1, end=35, tick=0.5),
        ygrid=dict(start=-0.005, end=0.15, tick=0.005),
        start=samples.value[0],
        end=samples.value[1]
    )
    
    with fw.batch_update():
        fw.data[0].x = data['x']
        fw.data[0].y = data['y']
        fw.data[0].z = data['z']
        
# observers
samples.observe(response, names="value")
selector.observe(response, names="value")

# update figure widget with callback
response(dict())

# create widgets
widgets.VBox([selector, fw, samples])

### Wheel: Movement

In [ ]:
# decleration selector
selector = widgets.Dropdown(
    options=[('Manuel', 1), ('Estimator', 2)],
    value=1,
    description='limit:'
)

# translational deceleration limit slider
limit = widgets.FloatSlider(
    value=measurement.a_mean, 
    min=-2, 
    max=0, 
    step=0.01, 
    description="limit",
    layout=widgets.Layout(width='50%'),
    continuous_update=False)

# samples range slider
samples = widgets.IntRangeSlider(
    value=[355, 1300],
    min=0, 
    max=len(rows), 
    step=5, 
    description="samples",
    layout=widgets.Layout(width='80%'),
    continuous_update=False)

# figure widget
fw = go.FigureWidget(make_subplots(
    rows=8, 
    cols=1,
    specs=[
        [{"secondary_y": True}], 
        [{"secondary_y": True}],
        [{"secondary_y": True}], 
        [{"secondary_y": True}], 
        [{"secondary_y": True}], 
        [{"secondary_y": True}], 
        [{"secondary_y": True}], 
        [{"secondary_y": True}]
    ],
    shared_xaxes=True,
    row_heights = (1.5, 0.5, 1, 1.5, 1.25, 1, 1, 1),
    vertical_spacing=0.01,
    x_title='samples'
).add_trace(
    measurement.speed.relabel('Vehicle speed [km/h]').plot(), 
    row=1, col=1,
).add_trace(
    velocity.relabel('Velocity [km/h]').plot(), 
    row=1, col=1, 
).add_trace(
    speed.level.relabel('Speed [km/h]').plot(), 
    row=1, col=1, 
).add_trace(
    pressure.level.relabel('Pressure [bar]').plot(fill='tozeroy'), 
    row=1, col=1,
    secondary_y=True
).add_trace(
    slip.div(speed.level - slip).mul(100).relabel('Relative Slip [%]').plot(),
    row=2, col=1,
).add_trace(
    slip.relabel('Slip [km/h]').plot(fill='tozeroy'), 
    row=2, col=1,
    secondary_y=True
).add_trace(
    adhesion.level.relabel('Adhesion [1]').plot(fill='tozeroy'), 
    row=3, col=1,
).add_trace(
    adhesion.trend.relabel('Adhesion-Trend').plot(fill='tozeroy'), 
    row=3, col=1,
    secondary_y=True
).add_trace(
    adhesion.trend.left_positive().move(-6).relabel('Adhesion-Maximum').digital.plot(), 
    row=3, col=1,
).add_trace(
    amplitude.relabel('Amplitude').plot(), 
    row=4, col=1,
).add_trace(
    vector.distance.relabel('Euclidean Distance').plot(), 
    row=4, col=1,
).add_trace(
    vector.dot.relabel('Dot Product').plot(), 
    row=4, col=1,
).add_trace(
    phase.relabel('Phase').plot(fill='tozeroy'), 
    row=4, col=1,
    secondary_y=True
).update_yaxes(
    range=[-180, 180],
    tickvals=[-180, -135, -90, -45, 0, 45, 90, 135, 180],
    row=4, col=1,
    secondary_y=True
).add_trace(
    state2d(acceleration.level, jerk.level, limit.value).digital.plot(fill='tozeroy'), 
    row=5, col=1,
).update_yaxes(
    tickvals=machine.numbers,
    ticktext=machine.labels,
    row=5, col=1,
    secondary_y=False
).add_trace(
    acceleration.level.relabel('Accelertation [m/s^2]').plot(), 
    row=6, col=1,
).add_trace(
    acceleration.level.sub(limit.value).relabel('Accelertation-a0 [m/s^2]').plot(fill='tozeroy'), 
    row=6, col=1,
).add_trace(
    acceleration.level.sub(limit.value).sign().relabel('Sign acceleration').digital.plot(), 
    row=6, col=1, 
    secondary_y=True
).add_trace(
    deceleration.relabel('Deceleration [m/s^2]').plot(), 
    row=6, col=1,
    secondary_y=False
).add_trace(
    estimator.deceleration.relabel('Translational Deceleration [m/s^2]').plot(), 
    row=6, col=1,
    secondary_y=False
).add_trace(
    jerk.level.relabel('Jerk [m/s^3]').plot(fill='tozeroy'), 
    row=7, col=1,
).add_trace(
    jerk.level_sign.relabel('Sign jerk').digital.plot(), 
    row=7, col=1, 
    secondary_y=True
).add_trace(
    snap.level.relabel('Snap [m/s^4]').plot(fill='tozeroy'), 
    row=8, col=1, 
).add_trace(
    snap.level_sign.relabel('Sign snap').digital.plot(), 
    row=8, col=1, 
    secondary_y=True
).update_layout(
    title='Wheel Movement',
    height=1800,
    margin=dict(t=50, b=60, l=50, r=50),
    #hovermode='x unified',
).update_traces(
    mode='lines'
))

# callback
def response(change):
    """ Callback to update the figure widget. """
    if selector.value == 2:
        _limit = estimator.deceleration
        limit.disabled = True
        _slip_absolute = estimator.slip
        _slip_relative = estimator.relative_slip        
    else:
        limit.disabled = False
        _limit = Trace('Translational Deceleration [m/s^2]', [limit.value] * len(estimator.deceleration))
        _slip_absolute = speed.trend.sub(_limit * 3.6 * dt).sums_negative()
        _slip_relative = _slip_absolute.div(speed.level - _slip_absolute).mul(100)

    # re-compute vector
    _vector = VectorTraces(x=acceleration.level - _limit, y=jerk.level)
    
    with fw.batch_update():
        fw.data[0].y = measurement.speed.samples[samples.value[0]:samples.value[1]]
        fw.data[1].y = speed.level.sub(_slip_absolute).samples[samples.value[0]:samples.value[1]]
        fw.data[2].y = speed.level.samples[samples.value[0]:samples.value[1]]
        fw.data[3].y = pressure.level.samples[samples.value[0]:samples.value[1]]
        fw.data[4].y = _slip_relative.samples[samples.value[0]:samples.value[1]]
        fw.data[5].y = _slip_absolute.samples[samples.value[0]:samples.value[1]]
        fw.data[6].y = adhesion.level.samples[samples.value[0]:samples.value[1]]
        fw.data[7].y = adhesion.trend.samples[samples.value[0]:samples.value[1]]
        fw.data[8].y = adhesion.trend.left_positive().move(-6).mul(0.1).samples[samples.value[0]:samples.value[1]]
        fw.data[9].y = _vector.r.samples[samples.value[0]:samples.value[1]]
        fw.data[10].y = _vector.distance.samples[samples.value[0]:samples.value[1]]
        fw.data[11].y = _vector.dot.mul(0.1).samples[samples.value[0]:samples.value[1]]
        fw.data[12].y = _vector.phi.mul(180 / math.pi).samples[samples.value[0]:samples.value[1]]
        fw.data[13].y = state2d(acceleration.level, jerk.level, _limit).samples[samples.value[0]:samples.value[1]]
        fw.data[14].y = acceleration.level.samples[samples.value[0]:samples.value[1]]
        fw.data[15].y = acceleration.level.sub(_limit).samples[samples.value[0]:samples.value[1]]
        fw.data[16].y = acceleration.level.sub(_limit).sign().samples[samples.value[0]:samples.value[1]]
        fw.data[17].y = deceleration.samples[samples.value[0]:samples.value[1]]
        fw.data[18].y = _limit[samples.value[0]:samples.value[1]]
        fw.data[19].y = jerk.level.samples[samples.value[0]:samples.value[1]]
        fw.data[20].y = jerk.level_sign.samples[samples.value[0]:samples.value[1]]
        fw.data[21].y = snap.level.samples[samples.value[0]:samples.value[1]]
        fw.data[22].y = snap.level_sign.samples[samples.value[0]:samples.value[1]]
        
# observers
selector.observe(response, names="value")
limit.observe(response, names="value")
samples.observe(response, names="value")

# update figure widget with callback
response(dict())

# create widgets
widgets.VBox([selector, limit, samples, fw])

### Wheel: Jerk-Acceleration Point-Space

3D-Scatter plot to display the jerk, acceleration points of the measured wheel speed speed with visualized wheel movement quadrants.

In [ ]:
# cutout to display
x1 = 355
x2 = 1300

# data to display
data = dict(
    x=rows[x1:x2],
    y=jerk.level[x1:x2],
    z=acceleration.level[x1:x2]
)

# radii
color = list(map(lambda point: math.sqrt(point[0]**2 + point[1]**2), zip(data['y'], data['z'])))

# translational deceleration limit
z0 = measurement.a_mean

# extemas
y_max = max(data['y'])
y_min = min(data['y'])
z_max = max(data['z'])
z_min = min(data['z'])

# figure
go.Figure([
    Cube('Quadrant1', 
         [x1, x1, x1, x1, x2, x2, x2, x2],
         [0, 0, y_min, y_min, 0, 0, y_min, y_min], 
         [z0, z_min, z_min, z0, z0, z_min, z_min, z0], 
         '#CD5C5C').plot(),
    Cube('Quadrant2',
         [x1, x1, x1, x1, x2, x2, x2, x2],
         [0, 0, y_max, y_max, 0, 0, y_max, y_max],  
         [z0, z_min, z_min, z0, z0, z_min, z_min, z0],
         '#FFA500').plot(),
    Cube('Quadrant3',
         [x1, x1, x1, x1, x2, x2, x2, x2],
         [0, 0, y_max, y_max, 0, 0, y_max, y_max], 
         [z0, z_max, z_max, z0, z0, z_max, z_max, z0], 
         '#1E90FF').plot(),
    Cube('Quadrant4',
         [x1, x1, x1, x1, x2, x2, x2, x2],
         [0, 0, y_min, y_min, 0, 0, y_min, y_min],
         [z0, z_max, z_max, z0, z0, z_max, z_max, z0], 
         '#20B2AA').plot(),
    go.Scatter3d(
        name='Acceleration',
        x=data['x'], 
        y=[int(max(jerk.level[x1:x2])) + 0.25] * len(data['x']), 
        z=acceleration.level[x1:x2], 
        mode='lines',
        line=dict(width=5)),
    go.Scatter3d(
        name='Jerk',
        x=data['x'], 
        y=jerk.level[x1:x2], 
        z=[int(min(acceleration.level[x1:x2])) - 0.25] * len(data['x']), 
        mode='lines',
        line=dict(width=5)),
    go.Scatter3d(
        name='Jerk-Accelaration Points',
        x=data['x'], 
        y=data['y'], 
        z=data['z'],
        mode='lines+markers',
        marker=dict(
            size=5,
            color=color,
            colorscale='Viridis',   
            opacity=0.75)),
    go.Scatter3d(
        name='Translational Deceleration Limit',
        x=data['x'], 
        y=[0] * len(data['x']), 
        z=estimator.deceleration[x1:x2], 
        mode='lines',
        line=dict(width=5)),
]).update_layout(
    title="Wheel: Jerk-Acceleration Points",
    height=1000,
    margin=dict(t=50, b=50, l=50, r=50),
    legend_orientation="h",
    scene_aspectratio=dict(x=2, y=1, z=1),
    scene_xaxis=dict(title='samples'),
    scene_yaxis=dict(
        title='Jerk [m/s^3]', 
        range=[math.floor(y_min - 0.5), math.ceil(y_max + 0.5)]),
    scene_zaxis=dict(
        title='Acceleration [m/s^2]', 
        range=[math.floor(z_min - 0.5), math.ceil(z_max + 0.5)]),
    scene_camera=dict(
        eye=dict(x=-0.25, y=-2, z=0.3))
).update_traces(
    showlegend=True
)

### Wheel: Jerk-Acceleration Vector-Space

3D-Cone plot to display the acceleration, jerk vectors of the measured wheel speed with visualized wheel movement quadrants.

In [ ]:
# cutout to display
x1 = 355
x2 = 1300

# data to display
data = dict(
    x=rows[x1:x2],
    y=jerk.level[x1:x2],
    z=acceleration.level[x1:x2],
    u=[1] * len(rows[x1:x2]),
    v=jerk.level.delta()[x1:x2],
    w=acceleration.level.delta()[x1:x2]
)

# translational deceleration limit
z0 = measurement.a_mean

# extremas
y_max = max(data['y'])
y_min = min(data['y'])
z_max = max(data['z'])
z_min = min(data['z'])

# figure
go.Figure([
    Cube('Quadrant1', 
         [x1, x1, x1, x1, x2, x2, x2, x2],
         [0, 0, y_min, y_min, 0, 0, y_min, y_min], 
         [z0, z_min, z_min, z0, z0, z_min, z_min, z0], 
         '#CD5C5C').plot(),
    Cube('Quadrant2',
         [x1, x1, x1, x1, x2, x2, x2, x2],
         [0, 0, y_max, y_max, 0, 0, y_max, y_max],  
         [z0, z_min, z_min, z0, z0, z_min, z_min, z0],
         '#FFA500').plot(),
    Cube('Quadrant3',
         [x1, x1, x1, x1, x2, x2, x2, x2],
         [0, 0, y_max, y_max, 0, 0, y_max, y_max], 
         [z0, z_max, z_max, z0, z0, z_max, z_max, z0], 
         '#1E90FF').plot(),
    Cube('Quadrant4',
         [x1, x1, x1, x1, x2, x2, x2, x2],
         [0, 0, y_min, y_min, 0, 0, y_min, y_min],
         [z0, z_max, z_max, z0, z0, z_max, z_max, z0], 
         '#20B2AA').plot(),
    go.Scatter3d(
        name='Acceleration',
        x=data['x'], 
        y=[int(max(jerk.level[x1:x2])) + 0.25] * len(data['x']), 
        z=acceleration.level[x1:x2], 
        mode='lines',
        line=dict(width=5)),
    go.Scatter3d(
        name='Jerk',
        x=data['x'], 
        y=jerk.level[x1:x2], 
        z=[int(min(acceleration.level[x1:x2])) - 0.25] * len(data['x']), 
        mode='lines',
        line=dict(width=5)),
    go.Cone(
        name='Jerk-Acceleration Vectors',
        x=data['x'],
        y=data['y'],
        z=data['z'],
        u=data['u'],
        v=data['v'],
        w=data['w'],
        sizemode="scaled",
        sizeref=3,
        anchor="tip"),
    go.Scatter3d(
        name='Translational Deceleration Limit',
        x=data['x'], 
        y=[0] * len(data['x']), 
        z=estimator.deceleration[x1:x2], 
        mode='lines',
        line=dict(width=5)),
]).update_layout(
    title="Wheel: Jerk-Acceleration Vectors",
    height=1000,
    margin=dict(t=50, b=50, l=50, r=50),
    legend_orientation="h",
    scene_aspectratio=dict(x=2, y=1, z=1),
    scene_xaxis=dict(title='samples'),
    scene_yaxis=dict(
        title='Jerk [m/s^3]', 
        range=[math.floor(y_min - 0.5), math.ceil(y_max + 0.5)]),
    scene_zaxis=dict(
        title='Acceleration [m/s^2]', 
        range=[math.floor(z_min - 0.5), math.ceil(z_max + 0.5)]),
    scene_camera=dict(
        eye=dict(x=-0.25, y=-2, z=0.3))
).update_traces(
    showlegend=True
)

### Wheel: Pressure-Acceleration Vectors

Quiver plot to display the pressure, acceleration vectors of the measured applied brake pressure and wheel speed.

In [ ]:
# samples range slider
samples = widgets.IntRangeSlider(
    value=[375, 375+150],
    min=0, 
    max=len(rows), 
    step=5, 
    description="samples",
    layout=widgets.Layout(width='80%'),
    continuous_update=True)

# figure widget
fw = go.FigureWidget(
    ff.create_quiver(
        name="Wheel: Pressure-Acceleration Vectors",
        x=pressure.level.samples, 
        y=acceleration.level.samples, 
        u=pressure.level.delta().samples, 
        v=acceleration.level.delta().samples,
        scale=0.75,
        scaleratio = 0.25,
        arrow_scale=0.5,
        line_width=2).data[0]
).update_layout(
    title="Wheel: Pressure-Acceleration Vectors",
    height=800,
    margin=dict(t=50, b=50, l=50, r=50),
    legend_orientation="h",
    xaxis=dict(title="Pressure [bar]"),
    yaxis=dict(title="Acceleration [m/s^2]"),
).update_traces(
    showlegend=True
)

# callback
def response(change):
    """ Callback to update the figure widget. """
    
    # compute indexes of the arrow points
    indexes = range(samples.value[0], samples.value[1])
    indexes = chain(
        # indexes for arrow stem points
        chain.from_iterable(map(lambda x: repeat(x, 3), indexes)),
        # indexes for arrow tip points
        chain.from_iterable(map(lambda x: repeat(x, 4), indexes))) 
    
    # re-create quiver figure
    quiver = ff.create_quiver(
                x=pressure.level[samples.value[0]:samples.value[1]], 
                y=acceleration.level[samples.value[0]:samples.value[1]], 
                u=pressure.level.delta()[samples.value[0]:samples.value[1]], 
                v=acceleration.level.delta()[samples.value[0]:samples.value[1]],
                scale=0.75,
                scaleratio = 0.25,
                arrow_scale=0.5,
                text=[f'index: {i}' for i in indexes],
                line_width=2)
    
    # update quiver plot
    fw.data[0].update(quiver.data[0])
 
# observers
samples.observe(response, names="value")

# update figure widget with callback
response(dict())

# create widgets
widgets.VBox([fw, samples])

### Wheel: Pressure-Slip-Adhesion Vectors

3D-Cone plot to display the pressure, slip, adhesion vectors.

In [ ]:
# samples range slider
samples = widgets.IntRangeSlider(
    value=[360, 1300],
    min=0, 
    max=len(rows), 
    step=5, 
    description="samples",
    layout=widgets.Layout(width='80%'),
    continuous_update=True)

# figure widget
fw = go.FigureWidget([
    go.Cone(
        name='Pressure-Slip-Adhesion Vectors',
        x=pressure.level.samples,
        y=slip.samples,
        z=adhesion.level.samples,
        u=pressure.level.delta().samples,
        v=slip.delta().samples,
        w=adhesion.level.delta().samples,
        sizemode="scaled",
        sizeref=3,
        anchor="tip"),
]).update_layout(
    title="Wheel: Pressure-Slip-Adhesion Vectors",
    height=900,
    # width=1000,
    margin=dict(t=50, b=50, l=50, r=50),
    legend_orientation="h",
    scene_aspectratio=dict(x=1, y=1, z=0.9),
    scene_xaxis=dict(title='Brake Pressure [bar]'),
    scene_yaxis=dict(title='Relative Slip [%]'),
    scene_zaxis=dict(title='Adhesion [1]'),
    # camera settings
    scene_camera=dict(
        eye=dict(x=2, y=-1, z=0.5))
).update_traces(
    showlegend=True
)

# callback
def response(change):
    """ Callback to update the figure widget. """
    # relative slip [%]
    _slip = slip.div(speed.level - slip).mul(-100)
    
    # update data to display
    data = dict(
        x=pressure.level[samples.value[0]:samples.value[1]],
        y=_slip[samples.value[0]:samples.value[1]],
        z=adhesion.level[samples.value[0]:samples.value[1]],
        u=pressure.level.delta()[samples.value[0]:samples.value[1]],
        v=_slip.delta()[samples.value[0]:samples.value[1]],
        w=adhesion.level.delta()[samples.value[0]:samples.value[1]]
    )

    # extremas
    x_max = max(data['x'])
    x_min = min(data['x'])
    y_max = max(data['y'])
    y_min = min(data['y'])
    z_max = max(data['z'])
    z_min = min(data['z'])
    
    with fw.batch_update():
        # update cone
        fw.data[0].x = data['x']
        fw.data[0].y = data['y']
        fw.data[0].z = data['z']
        fw.data[0].u = data['u']
        fw.data[0].v = data['v']
        fw.data[0].w = data['w']
        fw.data[0].text = [f'index: {i}' for i in range(samples.value[0], samples.value[1])]
    
    # update layout
    fw.update_layout(
        scene_xaxis=dict(
            range=[x_min, x_max]),
        scene_yaxis=dict(
            range=[y_min, y_max]),
        scene_zaxis=dict(
            range=[z_min, z_max]))

# observers
samples.observe(response, names="value")

# update figure widget with callback
response(dict())

# create widgets
widgets.VBox([fw, samples])

### Wheel: Acceleration-Slip-Pressure Vectors

3D-Cone plot to display the acceleration, slip, pressure vectors.

In [ ]:
# samples range slider
samples = widgets.IntRangeSlider(
    value=[360, 525],
    min=0,
    max=len(rows),
    step=5,
    description="samples",
    layout=widgets.Layout(width='80%'),
    continuous_update=True)

# figure widget
fw = go.FigureWidget([
    go.Cone(
        name='Acceleration-Slip-Pressure Vectors',
        x=acceleration.level.samples,
        y=slip.samples,
        z=pressure.level.samples,
        u=acceleration.level.delta().samples,
        v=slip.delta().samples,
        w=pressure.level.delta().samples,
        sizemode="scaled",
        sizeref=2,
        anchor="tip"),
]).update_layout(
    title="Wheel: Acceleration-Slip-Pressure Vectors",
    height=1000,
    margin=dict(t=50, b=50, l=50, r=50),
    legend_orientation="h",
    scene_aspectratio=dict(x=1, y=1, z=1),
    scene_xaxis=dict(title='Acceleration [m/s^2]'),
    scene_yaxis=dict(title='Relative Slip [%]'),
    scene_zaxis=dict(title='Brake Pressure [bar]'),
    # camera settings
    scene_camera=dict(
        eye=dict(x=1.5, y=-1.5, z=1.5))
).update_traces(
    showlegend=True
)

# callback
def response(change):
    """ Callback to update the figure widget. """
    # relative slip [%]
    _slip = slip.div(speed.level - slip).mul(-100)
    
    # update data to display
    data = dict(
        x=acceleration.level[samples.value[0]:samples.value[1]],
        y=_slip[samples.value[0]:samples.value[1]],
        z=pressure.level[samples.value[0]:samples.value[1]],
        u=acceleration.level.delta()[samples.value[0]:samples.value[1]],
        v=_slip.delta()[samples.value[0]:samples.value[1]],
        w=pressure.level.delta()[samples.value[0]:samples.value[1]]
    )

    # extremas
    x_max = max(data['x'])
    x_min = min(data['x'])
    y_max = max(data['y'])
    y_min = min(data['y'])
    z_max = max(data['z'])
    z_min = min(data['z'])

    with fw.batch_update():
        # update cone
        fw.data[0].x = data['x']
        fw.data[0].y = data['y']
        fw.data[0].z = data['z']
        fw.data[0].u = data['u']
        fw.data[0].v = data['v']
        fw.data[0].w = data['w']
        fw.data[0].text = [f'index: {i}' for i in range(samples.value[0], samples.value[1])]
    
    # update layout
    fw.update_layout(
        scene_xaxis=dict(
            range=[x_min, x_max]),
        scene_yaxis=dict(
            range=[y_min, y_max]),
        scene_zaxis=dict(
            range=[z_min, z_max]))

# observers
samples.observe(response, names="value")

# update figure widget with callback
response(dict())

# create widgets
widgets.VBox([fw, samples])